In [1]:
%load_ext autoreload
%autoreload 2
 # then imported files are u pdated each time a cell is executed (good if functions in py files are added and modified)

import scipy.io
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import pickle
import sys
import os

os.add_dll_directory("C:/OpenSim4.4/bin") # otherwise module _sombody not found error!

import opensim as osim
from scipy.spatial.transform import Rotation as R

sys.path.insert(0, '..')
sys.path.insert(0, '...')

from OsimPython.plotUtil import addArrow, addMarker
from OsimPython.osim_type_util import osimVec3ToArray

from Linalg.ray_tracing_util import intersectionPoint_pln_line, in_hull, dist_lines, get_regressionPlane
from Paper_util.IROS_2024_LMC.LMC_util import plotLMC, make_LMC, getLMC_plns, check_intersection_finger, check_intersection_palm, check_LMC_Hand_visibility

from OsimPython.Hand_model import getHandModel, getHandMarkers, setHandPose, plot_hand

#paper_path = 'D:\Dokumente\Projects\HULD\Paper_util/IROS_2024_LMC/'

In [2]:
HandModel = getHandModel()
q = np.zeros(20)
# set the thumb in a flat position
q[0] = 0.4
q[1] = -0.6
q[2] = 0.1
state = setHandPose(HandModel, q)
# HAND ROTATION
# x = + flexion, - extension
# y = + pronation, - supination
# z = + ulna devation, - radial deviation
pos_markers = getHandMarkers(HandModel, state, pos=np.array([0, 0, 0]), orient=np.array([0, 0, 0])*np.pi/180) 

print(pos_markers[24] - pos_markers[25])

fig = go.Figure()
plot_hand(None, pos_markers).show()

[5.20087260e+01 1.87555610e+00 1.86423717e-04]


In [3]:
fig = go.Figure()

LMC_H = 600
LMC0 = make_LMC(LMC_loc=[0,0,0], LMC_orient=[0, 0, -45], LMC_H=LMC_H)
LMC1 = make_LMC(LMC_loc=[50,0,0], LMC_orient=[-20, 0, 45], LMC_H=LMC_H)
LMC2 = make_LMC(LMC_loc=[50,50,0], LMC_orient=[20, 0, -45], LMC_H=LMC_H)
LMC3 = make_LMC(LMC_loc=[0,50,0], LMC_orient=[20, 0, 45], LMC_H=LMC_H)

fig = plotLMC(fig, LMC0, color='red', name='0')
fig = plotLMC(fig, LMC1, color='green', name='1')
fig = plotLMC(fig, LMC2, color='blue', name='2')
fig = plotLMC(fig, LMC3, color='orange', name='3')

fig.show()

In [ ]:
# GENERAL
HandModel = getHandModel()
q = np.zeros(20)
# set the thumb in a flat position
q[0] = 0.4
q[1] = -0.6
q[2] = 0.1
state = setHandPose(HandModel, q)
fingers_idx = np.array([[0,3,8,17], #thumb (root to tip)
                        [4,9,13,18], #index
                        [5,10,14,19], #middle
                        [6,11,15,20], #ring
                        [7,12,16,21]]) #pinky
finger_radius = 10 # mm
#palm_plane_idx = np.array([7, 6, 5, 4, 1, 25, 24, 2]) # cirsumverence of palm clockwise

# HAND
# x = + flexion, - extension
# y = + pronation, - supination
# z = + ulna devation, - radial deviation
pos_markers = getHandMarkers(HandModel, state, pos=np.array([0, -100, 300]), orient=np.array([0., 0., 0.]))

# PALM;
pv__ = (pos_markers[2] + pos_markers[1]) / 2 - pos_markers[6] # vector pointing from the finger root to the wrist
pv__ /= np.linalg.norm(pv__)
palm_markers = np.array([pos_markers[7] + pv__*3, pos_markers[6] + pv__*3, pos_markers[5] + pv__*3, pos_markers[4] + pv__*3, # to make sure the finger root markers do not intersect with the palm
                         pos_markers[1], pos_markers[25], pos_markers[24], pos_markers[2]])
# normal plane to the palm and palm centroid
palm_plane_normal = get_regressionPlane(palm_markers)
palm_centroid = np.mean(palm_markers, axis=0)

# FINGERS
# the line chain of a finger starts at the finger root and clinbs marker by marker to the finger tip
# the base of a line is always the origin of a vector pointing from towards the tip -> this means the only frogner marker not a line origin is the finger tip
#finger_lines has the shape (num_fingers, num_phlanges, len_phlanges)
finger_lines = np.array([[(pos_markers[fingers_idx[j, i]], (pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]])/np.linalg.norm(pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]]), np.linalg.norm(pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]])) for i in range(fingers_idx.shape[1]-1)] for j in range(fingers_idx.shape[0])])

# LMC
LMC = make_LMC(LMC_loc=[0, 0, 0], LMC_orient=[0, 0, 0], LMC_H=600)

fig = go.Figure()
fig = plotLMC(fig, LMC, color='orange', name='LMC', scale = 30)
fig = plot_hand(fig, pos_markers)

occlusions = check_LMC_Hand_visibility(LMC, pos_markers, fingers_idx, finger_lines, finger_radius, palm_plane_normal, palm_centroid, palm_markers)
print(occlusions)
fig.show()

In [ ]:
def generate_hand_poses(num_poses, limitsPS=[180, -180], limis_FE=[60, 60], limits_WD=[40, 20]):
    # x = + flexion, - extension
    # y = + pronation, - supination
# z = + ulna devation, - radial deviation

    fingers_idx = np.array([[0,3,8,17], #thumb (root to tip)
                        [4,9,13,18], #index
                        [5,10,14,19], #middle
                        [6,11,15,20], #ring
                        [7,12,16,21]]) #pinky

    handMarkers = np.zeros([num_poses, 26, 3])
    palmMarkers = np.zeros([num_poses, 8, 3])
    palmPlnNormals = np.zeros([num_poses, 3])
    palmCentroids = np.zeros([num_poses, 3])
    fingers = []

    ax = np.random.uniform(limis_FE[0],limis_FE[1],num_poses) * np.pi/180
    ay = np.random.uniform(limitsPS[0],limitsPS[1],num_poses) * np.pi/180
    az = np.random.uniform(limits_WD[0],limits_WD[1],num_poses) * np.pi/180

    for i in range(num_poses):
        pos_markers = getHandMarkers(HandModel, state, pos=np.array([0, -100, 300]), orient=np.array([ax[i], ay[i], az[i]]))
        pos_markers *= 1/np.linalg.norm(pos_markers[20]-pos_markers[24]) * 200 # scale hand to 200mm length
        handMarkers[i, :] = pos_markers
        pv__ = (pos_markers[2] + pos_markers[1]) / 2 - pos_markers[6] # vector pointing from the finger root to the wrist
        pv__ /= np.linalg.norm(pv__)
        palmMarkers[i, :] = np.array([pos_markers[7] + pv__*3, pos_markers[6] + pv__*3, pos_markers[5] + pv__*3, pos_markers[4] + pv__*3, # to make sure the finger root markers do not intersect with the palm
                            pos_markers[1], pos_markers[25], pos_markers[24], pos_markers[2]])

        palmPlnNormals[i, :] = get_regressionPlane(palmMarkers[i, :]) # normal plane to the palm and palm centroid
        palmCentroids[i, :] = np.mean(palmMarkers[i, :], axis=0) # centroid marker of the palm

        # FINGERS
        # the line chain of a finger starts at the finger root and clinbs marker by marker to the finger tip
        # the base of a line is always the origin of a vector pointing from towards the tip -> this means the only frogner marker not a line origin is the finger tip
        #finger_lines has the shape (num_fingers, num_phlanges, len_phlanges)
        fingers += [np.array([[(pos_markers[fingers_idx[j, i]], 
                                (pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]])/np.linalg.norm(pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]]), 
                                np.linalg.norm(pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]])) for i in range(fingers_idx.shape[1]-1)] for j in range(fingers_idx.shape[0])], dtype=object)]
    
    return handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers

def generate_hand_pose(pos, PS, FE, WD):
    # x = + flexion, - extension
    # y = + pronation, - supination
    # z = + ulna devation, - radial deviation

    fingers_idx = np.array([[0,3,8,17], #thumb (root to tip)
                        [4,9,13,18], #index
                        [5,10,14,19], #middle
                        [6,11,15,20], #ring
                        [7,12,16,21]]) #pinky

    handMarkers = np.zeros([1, 26, 3])
    palmMarkers = np.zeros([1, 8, 3])
    palmPlnNormals = np.zeros([1, 3])
    palmCentroids = np.zeros([1, 3])
    fingers = []

    i = 0
    pos_markers = getHandMarkers(HandModel, state, pos=pos, orient=np.array([FE, PS, WD])*np.pi/180)
    pos_markers *= 1/np.linalg.norm(pos_markers[20]-pos_markers[24]) * 200 # scale hand to 200mm length
    handMarkers[i, :] = pos_markers
    pv__ = (pos_markers[2] + pos_markers[1]) / 2 - pos_markers[6] # vector pointing from the finger root to the wrist
    pv__ /= np.linalg.norm(pv__)
    palmMarkers[i, :] = np.array([pos_markers[7] + pv__*3, pos_markers[6] + pv__*3, pos_markers[5] + pv__*3, pos_markers[4] + pv__*3, # to make sure the finger root markers do not intersect with the palm
                        pos_markers[1], pos_markers[25], pos_markers[24], pos_markers[2]])

    palmPlnNormals[i, :] = get_regressionPlane(palmMarkers[i, :]) # normal plane to the palm and palm centroid
    palmCentroids[i, :] = np.mean(palmMarkers[i, :], axis=0) # centroid marker of the palm

    # FINGERS
    # the line chain of a finger starts at the finger root and clinbs marker by marker to the finger tip
    # the base of a line is always the origin of a vector pointing from towards the tip -> this means the only frogner marker not a line origin is the finger tip
    #finger_lines has the shape (num_fingers, num_phlanges, len_phlanges)
    fingers += [np.array([[(pos_markers[fingers_idx[j, i]], 
                            (pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]])/np.linalg.norm(pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]]), 
                            np.linalg.norm(pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]])) for i in range(fingers_idx.shape[1]-1)] for j in range(fingers_idx.shape[0])], dtype=object)]
    
    return handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers

In [39]:
handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers = generate_hand_pose(pos=np.array([0, -100, 300]), PS=90, FE=0, WD=0)
data = [handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers]
LMCs = [make_LMC(LMC_loc=[-380,  130, 0], LMC_orient=[0, 0, 0], LMC_H=LMC_H) ]
i = 0
print(check_LMC_Hand_visibility(LMCs[0], pos_markers=data[0][i], fingers_idx=fingers_idx, finger_lines=data[4][0], finger_radius=finger_radius, palm_plane_normal=data[2][i], palm_centroid=data[3][i], palm_markers=data[1][i]))

fig = go.Figure()
fig = plotLMC(fig, LMCs[0], color='orange', name='LMC', scale = 30)
fig = plot_hand(fig, data[0][0])
fig.show()

[1. 1. 1. 1. 1. 1. 1.]


In [46]:
from scipy.optimize import minimize
# Input parameter
fingers_idx = np.array([[0,3,8,17], #thumb (root to tip)
                    [4,9,13,18], #index
                    [5,10,14,19], #middle
                    [6,11,15,20], #ring
                    [7,12,16,21]]) #pinky
    
finger_radius = 10 # mm
LMC_H = 600 # mm

num_LMC = 1
# Generate data
num_poses = 1
#handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers = generate_hand_poses(num_poses=num_poses, limitsPS=[0, 0], limis_FE=[0, 0], limits_WD=[0, 0])
handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers = generate_hand_pose(pos=np.array([0, -100, 300]), PS=90, FE=0, WD=0)


data = [handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers]

#x0 = np.zeros(4*num_LMC) # [lmcx, lmcy, lmc_xang, lmc_zang]
#x0 = np.random.rand(4*num_LMC) * np.array([10000, 10000, 90, 90]*num_LMC)

# optimization
def fun(x, args):
    #pos = x*np.array([300, 300, 90, 180]*num_LMC)
    pos = x*np.array([1000, 1000]*num_LMC)
    print(x)
    cost = []
    data = args
    fig = go.Figure()
    for xx in pos: # for each particle
        #LMCs = [make_LMC(LMC_loc=[xx[0+4*l], xx[1+4*l], 0], LMC_orient=[xx[2+4*l], 0, xx[3+4*l]], LMC_H=LMC_H) for l in range(num_LMC)]
        LMCs = [make_LMC(LMC_loc=[xx[0+4*l], xx[1+4*l], 0], LMC_orient=[0, 0, 0], LMC_H=LMC_H) for l in range(num_LMC)]
        occlusions = []
        for LMC in LMCs:
            for i in range(num_poses):
                occlusions += [check_LMC_Hand_visibility(LMC, pos_markers=data[0][i], fingers_idx=fingers_idx, finger_lines=data[4][0], finger_radius=finger_radius, palm_plane_normal=data[2][i], palm_centroid=data[3][i], palm_markers=data[1][i])]

        occlusions = np.array(occlusions) 
        print(occlusions)
        #print(np.sum(occlusions))
        # fig = go.Figure()
        # fig = plotLMC(fig, LMCs[0], color='orange', name='LMC', scale = 30)
        # fig = plot_hand(fig, data[0][0])
        # fig.show()
        
        fig = addMarker(fig, LMCs[0][0])
        met = (1-(occlusions[0, 0] * occlusions[0, 1])) * 10
        met += (5-np.sum(occlusions[0, 2:-1])) * 1
        #met += np.linalg.norm(data[0][0] - [xx[0], xx[1], 0])
        print(met)
        cost.append(met)
    fig.show()
    return cost

#print(fun(x0, data))

#res = minimize(fun, x0, args=(data), options=dict(disp = True))#, method='SLSQP')
from pyswarms.single.global_best import GlobalBestPSO

# instatiate the optimizer
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
# x_max = np.array([300, 300, 90, 180]*num_LMC)
# x_min = np.array([-300, -300, -90, -180]*num_LMC)
x_max = np.array([1, 1, 1, 1]*num_LMC)
x_min = np.array([-1, -1, -1, -1]*num_LMC)
x_max = np.array([1, 1]*num_LMC)
x_min = np.array([-1, -1]*num_LMC)
bounds = (x_min, x_max)
optimizer = GlobalBestPSO(n_particles=10, dimensions=2*num_LMC, options=options, bounds=bounds)

# now run the optimization, pass a=1 and b=100 as a tuple assigned to args
cost, pos = optimizer.optimize(fun, 1, args=(data))



2024-02-16 11:48:41,443 - pyswarms.single.global_best - INFO - Optimize for 1 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/1

[[ 0.37877868  0.13488567]
 [ 0.42021717 -0.2709991 ]
 [-0.03750125  0.33172913]
 [-0.02386014 -0.44324309]
 [ 0.9786535   0.51059903]
 [-0.63845133  0.46477027]
 [-0.30792435 -0.82502065]
 [-0.95825139 -0.70189191]
 [ 0.48212152 -0.51251185]
 [-0.75089713 -0.02468197]]
[[1. 1. 1. 1. 1. 1. 1.]]
1.0
[[0. 0. 0. 0. 0. 0. 0.]]
15.0
[[1. 1. 1. 1. 0. 0. 0.]]
3.0
[[0. 0. 0. 0. 0. 0. 0.]]
15.0
[[0. 0. 0. 0. 0. 0. 0.]]
15.0
[[0. 0. 0. 0. 0. 0. 0.]]
15.0
[[0. 0. 0. 0. 0. 0. 0.]]
15.0
[[0. 0. 0. 0. 0. 0. 0.]]
15.0
[[0. 0. 0. 0. 0. 0. 0.]]
15.0
[[0. 0. 0. 0. 0. 0. 0.]]
15.0


d:\Dokumente\Projects\HULD\Paper_util\IROS_2024_LMC\LMC_util.py:218: RuntimeWarning:

invalid value encountered in arccos



pyswarms.single.global_best: 100%|██████████|1/1, best_cost=1
2024-02-16 11:48:41,522 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.0, best pos: [0.37877868 0.13488567]


In [47]:
print(cost, pos)

1.0 [0.37877868 0.13488567]


In [49]:
x = pos * np.array([1000, 1000]*num_LMC)
print(x)
#x = np.array([300, 0, 0, 0])
LMCs = [make_LMC(LMC_loc=[x[0+4*l], x[1+4*l], 0], LMC_orient=[0, 0, 0], LMC_H=LMC_H) for l in range(num_LMC)]

occlusions = []
for LMC in LMCs:
    for i in range(num_poses):
        occlusions += [check_LMC_Hand_visibility(LMC, pos_markers=data[0][i], fingers_idx=fingers_idx, finger_lines=data[4][0], finger_radius=finger_radius, palm_plane_normal=data[2][i], palm_centroid=data[3][i], palm_markers=data[1][i])]

occlusions = np.array(occlusions)
print(occlusions)
fig = go.Figure()
fig = plotLMC(fig, LMCs[0], color='orange', name='LMC', scale = 30)
fig = plot_hand(fig, data[0][0])
fig.show()

[378.7786847  134.88566804]
[[1. 1. 1. 1. 1. 1. 1.]]


# TEsts

In [ ]:


t_p1 = np.array([-20, 100, 40])
t_p2 = np.array([40, 50, 0])

fig = plot_hand(pos_markers)
fig = plot_hand_and_marker(pos_markers, [0, 0, 0], fig=fig, color='black')
fig = plot_hand_and_marker(pos_markers, t_p1, fig=fig, color='green')
fig = plot_hand_and_marker(pos_markers, t_p2, fig=fig, color='green')
fig = plot_hand_and_marker(pos_markers, palm_centroid, fig=fig)
fig = addArrow(fig, palm_centroid, n*100, color='red')

v = t_p1 - t_p2
v = v / np.linalg.norm(v)
itsctpnt = intersectionPoint_pln_line(n, v, palm_centroid, t_p2)

print(in_hull(pos_markers[palm_plane_idx], itsctpnt))

fig = plot_hand_and_marker(pos_markers, itsctpnt, fig=fig, color='orange')
fig = addArrow(fig, t_p2, v*100, color='green')

# see if crosses a finger
# testpoints
pt1 = (pos_markers[10] + pos_markers[14]) / 2 + [0, -20, 30]
pt2 = (pos_markers[10] + pos_markers[14]) / 2 + [10, -20, -30]
v1 = pt1-pt2
v1 /= np.linalg.norm(v1)

fig = plot_hand_and_marker(pos_markers, pt1, fig=fig, color='pink', name='pt1')
fig = plot_hand_and_marker(pos_markers, pt2, fig=fig, color='cyan', name='pt2')
fig = addArrow(fig, pt2, v1*100, color='cyan')

v2 = pos_markers[14] - pos_markers[10]
v2 = v2 / np.linalg.norm(v2)

# nl = np.cross(ray, line)
# nl /= np.linalg.norm(nl)

# d = abs(nl @ (pos_markers[10] - p2)) / np.linalg.norm(nl)

# t1 = ((np.cross(line, nl)) @ (pos_markers[10] - p2)) / (nl @ nl)
# print(t1)
# print(d)

d, t1, t2 = dist_lines(pt2, pos_markers[10], v1, v2)

print(d, t1, t2)

fig = plot_hand_and_marker(pos_markers, pt2+v1*t1, fig=fig, color='cyan', name='closest')
fig = plot_hand_and_marker(pos_markers, pos_markers[10]+v2*t2, fig=fig, color='cyan', name='closest')
fig = addArrow(fig, pos_markers[10], v2*100, color='cyan')





fig.show()
